# Common imports

In [ ]:
#%matplotlib notebook
import os
import sys
import glob
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from PIL import Image
import random

# Provided functions
### Image loading and saving

In [ ]:
def LoadFaceImages(pathname, subject_name, num_images):
    """
    Load the set of face images.  
    The routine returns
        ambimage: image illuminated under the ambient lighting
        imarray: a 3-D array of images, h x w x Nimages
        lightdirs: Nimages x 3 array of light source directions
    """

    def load_image(fname):
        return np.asarray(Image.open(fname))

    def fname_to_ang(fname):
        yale_name = os.path.basename(fname)
        return int(yale_name[12:16]), int(yale_name[17:20])

    def sph2cart(az, el, r):
        rcos_theta = r * np.cos(el)
        x = rcos_theta * np.cos(az)
        y = rcos_theta * np.sin(az)
        z = r * np.sin(el)
        return x, y, z

    ambimage = load_image(
        os.path.join(pathname, subject_name + '_P00_Ambient.pgm'))
    im_list = glob.glob(os.path.join(pathname, subject_name + '_P00A*.pgm'))
    if num_images <= len(im_list):
        im_sub_list = np.random.choice(im_list, num_images, replace=False)
    else:
        print(
            'Total available images is less than specified.\nProceeding with %d images.\n'
            % len(im_list))
        im_sub_list = im_list
    im_sub_list.sort()
    imarray = np.stack([load_image(fname) for fname in im_sub_list], axis=-1)
    Ang = np.array([fname_to_ang(fname) for fname in im_sub_list])

    x, y, z = sph2cart(Ang[:, 0] / 180.0 * np.pi, Ang[:, 1] / 180.0 * np.pi, 1)
    lightdirs = np.stack([y, z, x], axis=-1)
    return ambimage, imarray, lightdirs

In [ ]:
def save_outputs(subject_name, albedo_image, surface_normals):
    im = Image.fromarray((albedo_image*255).astype(np.uint8))
    im.save("%s_albedo.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,0]*128+128).astype(np.uint8))
    im.save("%s_normals_x.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,1]*128+128).astype(np.uint8))
    im.save("%s_normals_y.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,2]*128+128).astype(np.uint8))
    im.save("%s_normals_z.jpg" % subject_name)

### Plot the height map

In [ ]:
def set_aspect_equal_3d(ax):
    """https://stackoverflow.com/questions/13685386"""
    """Fix equal aspect bug for 3D plots."""
    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()
    from numpy import mean
    xmean = mean(xlim)
    ymean = mean(ylim)
    zmean = mean(zlim)
    plot_radius = max([
        abs(lim - mean_)
        for lims, mean_ in ((xlim, xmean), (ylim, ymean), (zlim, zmean))
        for lim in lims
    ])
    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])
    ax.set_zlim3d([zmean - plot_radius, zmean + plot_radius])


def display_output(albedo_image, height_map):
    #fig = plt.figure()
    #plt.imshow(albedo_image, cmap='gray')
    #plt.axis('off')
    
    
    from importlib import reload
    reload(plt)
    %matplotlib notebook
    
    fig = plt.figure(figsize = (7, 7))
    ax = fig.gca(projection='3d')
    ax.view_init(20, 20)
    X = np.arange(albedo_image.shape[0])
    Y = np.arange(albedo_image.shape[1])
    X, Y = np.meshgrid(Y, X)
    H = np.flipud(np.fliplr(height_map))
    A = np.flipud(np.fliplr(albedo_image))
    A = np.stack([A, A, A], axis=-1)
    ax.xaxis.set_ticks([])
    ax.xaxis.set_label_text('Z')
    ax.yaxis.set_ticks([])
    ax.yaxis.set_label_text('X')
    ax.zaxis.set_ticks([])
    ax.yaxis.set_label_text('Y')
    surf = ax.plot_surface(
        H, X, Y, cmap='gray', facecolors=A, linewidth=0, antialiased=False)
    set_aspect_equal_3d(ax)

### Plot the surface norms. 

In [ ]:
def plot_surface_normals(surface_normals):
    """
    surface_normals: h x w x 3 matrix.
    """
    fig = plt.figure(figsize = (15, 8))
    ax = plt.subplot(1, 3, 1)
    ax.axis('off')
    ax.set_title('X')
    im = ax.imshow(surface_normals[:,:,0], 'jet', vmin = -1, vmax = 1)
    ax = plt.subplot(1, 3, 2)
    ax.axis('off')
    ax.set_title('Y')
    im = ax.imshow(surface_normals[:,:,1], 'jet', vmin = -1, vmax = 1)
    ax = plt.subplot(1, 3, 3)
    ax.axis('off')
    ax.set_title('Z')
    im = ax.imshow(surface_normals[:,:,2], 'jet', vmin = -1, vmax = 1)

# Your implementation

In [ ]:
def preprocess(ambimage, imarray):
    """
    preprocess the data: 
        1. subtract ambient_image from each image in imarray.
        2. make sure no pixel is less than zero.
        3. rescale values in imarray to be between 0 and 1.
    Inputs:
        ambimage: h x w
        imarray: h x w x Nimages
    Outputs:
        processed_imarray: h x w x Nimages
    """
    imarray_float = np.float64(imarray)
    ambient_image_float = np.float64(ambient_image)
    
    #subtract ambient_image from each image in imarray
    imarray_substracted = (imarray_float.transpose((2, 0, 1)) - ambient_image_float).transpose((1, 2, 0)) 
    
    imarray_substracted = imarray_substracted.clip(min = 0.) # make sure no pixel is less than zero
    
    processed_imarray=imarray_substracted/255.  #rescale values in imarray to be between 0 and 1  
    #Max - Min normalization, where min = 0, max = 1: x = (x-min)/(max-min)
    
    return processed_imarray

In [ ]:
def photometric_stereo(imarray, light_dirs):
    """
    Inputs:
        imarray:  h x w x Nimages
        light_dirs: Nimages x 3
    Outputs:
        albedo_image: h x w
        surface_norms: h x w x 3
    """
    imarray_stack = imarray.transpose(2, 0, 1).reshape((imarray.shape[2], imarray.shape[0]*imarray.shape[1]))
    #For each imarray.shape[0]*imarray.shape[1] pixel, 64*1 array stacked horizontally, i.e. 64*x array 
    #light_dirs - 64*3 array
    
    g_stacked = np.linalg.lstsq(light_dirs, imarray_stack, rcond=None)[0]
    #Least squares solution (3*x array - 3*1 for each pixel) 
    
    g_stacked_norm = np.linalg.norm(g_stacked, axis = 0)
    albedo_image = g_stacked_norm.reshape(imarray.shape[0], imarray.shape[1]) 
    #Maginitude of least square solution - reshaped to h*w
    
    surface_normals = (g_stacked/g_stacked_norm).T.reshape((imarray.shape[0], imarray.shape[1], 3))
    #Surface normals reshaped to h*w*3
    return albedo_image, surface_normals

In [ ]:
def get_surface(surface_normals, integration_method):
    """
    Inputs:
        surface_normals:h x w x 3
        integration_method: string in ['average', 'column', 'row', 'random']
    Outputs:
        height_map: h x w
    """
    
    #Calculating the partial derivatives f_x and f_y
    f_x = surface_normals[:,:,1]/surface_normals[:,:,2]
    f_y = surface_normals[:,:,0]/surface_normals[:,:,2]

    #Calculating the cumulative sums for the integrals 
    f_x_cumulative = np.cumsum(f_x, axis=0)
    f_y_cumulative = np.cumsum(f_y, axis=1)
    
    #Defining a column and a row of zeros for randomized method 
    zeros_y = np.zeros((f_y_cumulative.shape[0],1))
    zeros_x = np.zeros((1, f_x_cumulative.shape[1]))
    
    #For random method, computing a new f_y_cumulative and f_x_cumulative 
    f_y_cumulative_modified = np.copy(f_y_cumulative)
    f_x_cumulative_modified = np.copy(f_x_cumulative)
    
    integral_row= f_y_cumulative[0, :] + f_x_cumulative #Going horizontally forward and then vertically down 
    integral_col= f_x_cumulative[:, 0].reshape(-1,1) + f_y_cumulative #Going vertically down and then horizontally forward 
    
    if integration_method == 'row':
        height_map = integral_row
    
    if integration_method == 'column':
        height_map = integral_col
        
    if integration_method == 'average':
        height_map = 0.5*(integral_row + integral_col)  
        
    if integration_method == 'random':
        num_random_paths = 10000
        for i in range(num_random_paths): #Calculating integrals from 50 randomized paths and then averaging the results 
            
            #For random method, computing a new f_y_cumulative and f_x_cumulative 
            f_y_cumulative_modified = np.copy(f_y_cumulative)
            f_x_cumulative_modified = np.copy(f_x_cumulative)
            
            #Choosing a random x, y starting point 
            start_x, start_y = random.randint(0, f_x.shape[0]-1), random.randint(0, f_x.shape[1]-1)
            
            #Calculating the cumulative partial derivatives from the selected randomized point 
            #Nothing to be done for f_y_cumulative if starting y axis = 0
            if(start_y>0):
                #Calculting the cumulative sum after flipping 
                f_y_flip_cumulative = np.flip(np.flip(f_y[:, :start_y], axis = 1).cumsum(axis = 1), axis = 1)
                #Moving in the negative direction 
                f_y_cumulative_modified[:, :start_y] = f_y[:, start_y].reshape(-1,1) - f_y_flip_cumulative
                f_y_cumulative_modified[:, start_y:] = f_y_cumulative[:, start_y:] - f_y_cumulative[:, start_y-1].reshape(-1,1)
            
            #Nothing to be done for f_x_cumulative if starting x axis = 0
            if (start_x >0):   
                #Calculting the cumulative sum after flipping 
                f_x_flip_cumulative = np.flip(np.flip(f_x[:start_x, :], axis = 0).cumsum(axis = 0), axis = 0)
                #Moving in the negative direction 
                f_x_cumulative_modified[:start_x, :] = f_x[start_x, :] - f_x_flip_cumulative 
                f_x_cumulative_modified[start_x:, :] = f_x_cumulative[start_x:, :] - f_x_cumulative[start_x-1, :]
            
            #Choosing a random path < row wise - 0, column wise- 1>
            method = random.randint(0, 1)
            #Calculating the height map row_wise 
            if method ==0:
                integral_row_randomstart = f_y_cumulative_modified[start_x, :] + f_x_cumulative_modified
                
                if (i==0):
                    height_map=integral_row_randomstart
                else:
                    height_map+=integral_row_randomstart
            #Calculating the height map column_wise 
            if method ==1:
                integral_col_randomstart = f_x_cumulative_modified[:, start_y].reshape(-1,1) + f_y_cumulative_modified
                
                if (i==0):
                    height_map=integral_col_randomstart
                else:
                    height_map+=integral_col_randomstart
        #Averaging the height maps obtained in 50 randomized paths
        height_map/=num_random_paths
    return height_map

# Main function

In [ ]:
root_path = 'C:\\Users\\Rachneet Kaur\\Desktop\\UIUC\\UIUC Spring 2019\\CS 543 CV\\MP1\\croppedyale\\'
subject_name = 'yaleB05'
integration_method = 'average'
save_flag = True

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name,
                                                    64)
#ambient_image - x*y shaped image
#imarray - x*y*64 shaped 64 images 
#light_dirs - 64*3shaped 64 light directions 

processed_imarray = preprocess(ambient_image, imarray)
albedo_image, surface_normals = photometric_stereo(processed_imarray,
                                                   light_dirs)
avg = 0.
runs = 1
for i in range(runs):
    start_time = time.time()
    height_map = get_surface(surface_normals, 'average')
    avg+=(time.time() - start_time)
print ('Time taken to generate height map = ', avg/runs) 
if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)


In [ ]:
#Plotting the albedo image
fig = plt.figure()
plt.imshow(albedo_image, cmap='gray')
plt.axis('off')
plt.title('Albedo for yaleB07')
plt.show()

In [ ]:
#Plotting the surface normals 
plot_surface_normals(surface_normals)

In [ ]:
#Displaying the albedo image and the 3D integrated height map 
display_output(albedo_image, height_map)